In [14]:
import os
import re
from collections import defaultdict

# Configuration
SOURCE_ROOT = r"C:\Users\harold.noble\Desktop\RIC\app\frontend\src\routes" # Your specific path
EXTENSIONS = {'.svelte', '.ts', '.py'}

def get_file_content(file_path):
    """Extract relevant content based on file type."""
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()
        if file_path.endswith('.svelte'):
            match = re.search(r'<script[^>]*>(.*?)</script>', content, re.DOTALL)
            return match.group(1).strip() if match else ''
        return content

def extract_imports(file_path, content):
    """Extract import paths from file content."""
    imports = set()
    if file_path.endswith('.svelte') or file_path.endswith('.ts'):
        # Match TypeScript imports
        for match in re.finditer(r'import\s+.*?\s+from\s+[\'"](.+?)[\'"]', content):
            imp = match.group(1)
            if imp.startswith(('./', '../')) and any(imp.endswith(ext) for ext in EXTENSIONS):
                imports.add(imp)
    elif file_path.endswith('.py'):
        # Match Python imports
        for match in re.finditer(r'from\s+(.+?)\s+import|import\s+(.+?)(?:\s+as\s+|$)', content):
            imp = match.group(1) or match.group(2)
            if imp.startswith(('./', '../')):
                imports.add(imp + '.py' if not imp.endswith('.py') else imp)
    return imports

def resolve_import_path(base_path, imp):
    """Resolve relative import to absolute path."""
    dir_name = os.path.dirname(base_path)
    full_path = os.path.normpath(os.path.join(dir_name, imp))
    return full_path if os.path.exists(full_path) else None

def build_dependency_graph():
    """Build a graph of file dependencies."""
    file_imports = {}  # file -> set of files it imports
    imported_by = defaultdict(set)  # file -> set of files importing it
    file_contents = {}  # file -> content

    for dirpath, _, filenames in os.walk(SOURCE_ROOT):
        for filename in filenames:
            if not any(filename.endswith(ext) for ext in EXTENSIONS):
                continue
            file_path = os.path.join(dirpath, filename)
            relative_path = os.path.relpath(file_path, SOURCE_ROOT).replace('\\', '/')
            content = get_file_content(file_path)
            file_contents[relative_path] = content
            imports = extract_imports(file_path, content)
            file_imports[relative_path] = set()

            for imp in imports:
                resolved_path = resolve_import_path(file_path, imp)
                if resolved_path:
                    resolved_relative = os.path.relpath(resolved_path, SOURCE_ROOT).replace('\\', '/')
                    file_imports[relative_path].add(resolved_relative)
                    imported_by[resolved_relative].add(relative_path)

    return file_imports, imported_by, file_contents

def generate_group_files(file_imports, imported_by, file_contents):
    """Generate text files for each group, starting with least-used files."""
    # Calculate usage counts
    usage_counts = {file: len(imported_by.get(file, set())) for file in file_contents}
    all_files = set(file_contents.keys())
    unused_files = {f for f in all_files if usage_counts.get(f, 0) == 0}

    # Sort files by usage count (ascending) for group generation
    sorted_files = sorted(usage_counts.items(), key=lambda x: x[1])

    group_num = 1
    processed_files = set()

    # Generate groups starting with most-used files
    for root_file, count in sorted_files:
        if root_file in processed_files or count == 0:  # Skip unused for now
            continue
        importers = imported_by.get(root_file, set())
        if not importers:
            continue

        output_file = f"group_{group_num}.txt"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(f"## GROUP {group_num} - Starting with {root_file} (Imported by {count} files)\n\n")
            for importer in sorted(importers):
                f.write(f"### {importer}\n")
                if importer.endswith('.svelte'):
                    f.write("<script lang=\"ts\">\n")
                    f.write(f"{file_contents[importer]}\n")
                    f.write("</script>\n\n")
                else:
                    f.write(f"{file_contents[importer]}\n\n")
                processed_files.add(importer)
            processed_files.add(root_file)
        print(f"Generated {output_file}")
        group_num += 1

    # Handle unused files in a separate file
    if unused_files:
        with open("group_unused.txt", 'w', encoding='utf-8') as f:
            f.write("## GROUP - Unused Files (Imported by 0 files)\n\n")
            for file in sorted(unused_files - processed_files):
                f.write(f"### {file}\n")
                if file.endswith('.svelte'):
                    f.write("<script lang=\"ts\">\n")
                    f.write(f"{file_contents[file]}\n")
                    f.write("</script>\n\n")
                else:
                    f.write(f"{file_contents[file]}\n\n")
                processed_files.add(file)
        print(f"Generated group_unused.txt with {len(unused_files)} unused files")

    print(f"Processed {len(processed_files)} of {len(file_contents)} files in {group_num-1} groups")

# Main execution
print(f"Scanning {SOURCE_ROOT} for .svelte, .ts, and .py files...")
file_imports, imported_by, file_contents = build_dependency_graph()
generate_group_files(file_imports, imported_by, file_contents)

Scanning C:\Users\harold.noble\Desktop\RIC\app\frontend\src\routes for .svelte, .ts, and .py files...
Generated group_unused.txt with 39 unused files
Processed 39 of 39 files in 0 groups
